In [347]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [383]:
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('seaborn-dark-palette')
mpl.style.use('seaborn-whitegrid')

In [384]:
kwargs = dict(sep=" ", header=None)
DF_python = pd.read_csv('python_output', **kwargs)
DF_cpp = pd.read_csv('cpp_output', **kwargs)

Times withon callback

In [385]:
def time_stats(times, title):
    print('{}: mean = {:.4f}, median = {:.4f}, std = {:.4f}'
          .format(title, times.mean(), times.median(), times.std()))

In [386]:
times_within_callback_colnumber = 9

# The following mess of a code is due to the fact that ROS_INFO and rospy.loginfo
#  print out messages a bit differently
time_python = DF_python.loc[:, times_within_callback_colnumber]
time_cpp = DF_cpp.loc[:, times_within_callback_colnumber+1].apply(lambda x: float(x[:-4]))
time_python.hist(log=True)
time_cpp.hist(log=True)
plt.show()

time_stats(time_cpp, 'Cpp')
time_stats(time_python, 'Python')

TypeError: 'float' object has no attribute '__getitem__'

Because `ROS_INFO` produces additional white spaces, we need to extract the right columns:

In [387]:
def _extract_yaw_and_throttle_colnumbers(DF_data):
    one_row = DF_data.iloc[0]
    yaw_colnumber = one_row.index[one_row == 'yaw:']
    assert len(yaw_colnumber) == 1
    yaw_colnumber = yaw_colnumber[0]
    
    throttle_colnumber = one_row.index[one_row == 'throttle:']
    assert len(throttle_colnumber) == 1
    throttle_colnumber = throttle_colnumber[0]
    
    # "+1" because we want the columns following 'yaw:' and 'throttle:'
    colnumbers = [yaw_colnumber+1, throttle_colnumber+1]
    
    x_colname = one_row.index[one_row == 'x:']
    assert len(x_colname) == 1
    x_colname = x_colname[0]
    colnumbers = colnumbers + [x_colname+1]
    
    return DF_data.loc[:, colnumbers]


def _extract_timestamp(DF_data):
    one_row = DF_data.iloc[0]
    # I was getting NaN values -- hence the dropna
    # Also, I needed to cast everything to bool, because the NaNs imposed "object" type columns
    match_timestamp_regex = one_row.str.contains('^\[\w+.\w+\]:$').dropna().astype(bool)
    timestamp_colnumber = match_timestamp_regex.index[match_timestamp_regex]
    assert len(timestamp_colnumber) == 1
    timestamp_colnumber = timestamp_colnumber[0]
    return DF_data.loc[:, timestamp_colnumber].apply(lambda x: float(x[1:-2]))


def prepare_data(DF_data, suffix):
    DF_out = pd.DataFrame()
    DF_out[['yaw_'+suffix, 'throttle_'+suffix, 'x_'+suffix]] = _extract_yaw_and_throttle_colnumbers(DF_data)
    DF_out['timestamp'] = _extract_timestamp(DF_data)
    return DF_out

In [388]:
DF_python = prepare_data(DF_python, suffix='python')
DF_cpp = prepare_data(DF_cpp, suffix='cpp')

In [390]:
slc = slice(0, 3000)

ax = DF_python.iloc[slc].plot(x='timestamp', y='yaw_python')
ax = DF_cpp.iloc[slc].plot(x='timestamp', y='yaw_cpp', ax=ax)
plt.show()

ax = DF_python.iloc[slc].plot(x='timestamp', y='throttle_python')
ax = DF_cpp.iloc[slc].plot(x='timestamp', y='throttle_cpp', ax=ax)
plt.show()

ax = DF_python.iloc[slc].plot(x='timestamp', y='x_python')
ax = DF_cpp.iloc[slc].plot(x='timestamp', y='x_cpp', ax=ax)
plt.show()